<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

# écueils classiques

In [ ]:
import asyncio

## écueil n°1 :  fonction coroutine *vs* coroutine

In [ ]:
# une fonction coroutine
async def foo(delay):
    await asyncio.sleep(1)
    print("foo")

In [ ]:
# renvoie un objet coroutine
# si on l'appelle normalement
# il ne se passe rien
foo(4)

In [ ]:
# c'est exactement comme 
# une fonction génératrice
def squares(n):
    for i in range(n):
        print(i)
        yield i**2

In [ ]:
# qui retourne un
# itérateur, et là encore
# il ne se passe rien
squares(4)

### tous les scénarios

In [ ]:
def synchro():
    pass

In [ ]:
async def asynchro():
    pass

In [ ]:
def foo(): 
    synchro()        # 1 # OK
    asynchro()       # 2 # ** ATTENTION **
    await synchro()  # 3 # SyntaxError
    await asynchro   # 4 # SyntaxError

In [ ]:
async def afoo():
    synchro()        # 5 # OK
    await asynchro() # 6 # OK
    asynchro()       # 7 # ** ATTENTION **
    await synchro()  # 8 # ** ATTENTION **

### cas n°2

* une fonction appelle une coroutine sans `await`
* ➠ avertissement 

In [ ]:
!cat calls2.py

In [ ]:
!python calls2.py

### cas n°7

* une coroutine appelle une autre coroutine sans `await`
* idem : avertissement

In [ ]:
# avec until_complete
!cat calls7.py

In [ ]:
!python calls7.py

### cas n°8

In [ ]:
async def asynchro():
    await synchro()

##### protocole awaitable

| instruction | classe d'objets  | protocole | exemple | 
|-------------|------------------|-----------|---------|
|   `for`     |   itérables      |  `__iter__` | liste, ensemble |
|   `with`    | context managers | `__enter__` & `__exit__` | fichier |
|   `await`   |   awaitables     | `__await__` | objet coroutine |


In [ ]:
async def asynchro():
    await synchro()

* ***peut*** être légitime - si `synchro()` retourne un awaitable

* mais en général, c'est suspect !

# quel type de fonction ?

In [ ]:
from inspect import iscoroutinefunction
iscoroutinefunction(synchro)

In [ ]:
iscoroutinefunction(asynchro)

##### Attention toutefois

In [ ]:
# une vraie fonction qui renvoie un awaitable
iscoroutinefunction(asyncio.gather)

In [ ]:
# ditto
iscoroutinefunction(asyncio.wait)

## écueil n°2 : code trop bloquant

In [ ]:
async def countdown(n, period):
    while n >= 0:
        print('.', end='', flush=True)
        await asyncio.sleep(period)
        n -= 1

In [ ]:
import time
async def compute(n, period):
    for i in range(n):
        # on simule un calcul
        time.sleep(period)
        print('x', end='', flush=True)

In [ ]:
from asynchelpers import reset_loop
reset_loop()
asyncio.get_event_loop().run_until_complete(
    asyncio.gather(countdown(20, .1), compute(20, .05)))

### faites respirer votre code

In [ ]:
async def countdown(n, period):
    while n >= 0:
        #print(f"tick{n}")
        print('.', end='', flush=True)
        await asyncio.sleep(period)
        n -= 1

In [ ]:
import time
async def compute(n, period):
    for i in range(n):
        # on simule un calcul
        time.sleep(period)
        print('x', end='', flush=True)
        # await None n'est pas valide
        await asyncio.sleep(0)

In [ ]:
reset_loop()
asyncio.get_event_loop().run_until_complete(
    asyncio.gather(countdown(20, .1), compute(20, .05)))

# écueil n°3 

* exceptions non lues

In [ ]:
!cat raise.py

In [ ]:
# interrompre avec ii
!python raise.py

# bonnes pratiques de développement

* voir davantage de recettes de debug ici:
  https://docs.python.org/3/library/asyncio-dev.html

* notamment variable d'environnement `PYTHONASYNCIODEBUG`

# résumé

* bien utiliser `await` avec les coroutines
* appels synchrones: oui mais brefs 
* lire les exceptions une fois la boucle terminée
* penser à activer le mode debug en cas de souci